# Building Custom Reviewers

In this notebook, we will show how you can build custom reviewer agents using the LatteReview package.

## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [3]:
import json
import pandas as pd 
from pydantic import BaseModel
from typing import List, Dict, Any, Optional

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import BasicReviewer
from lattereview.workflows import ReviewWorkflow

## Data
For this notebook, we will use a real dataset that we collected years ago while conducting a literature review on AI applications in cardiothoracic imaging. Specifically, we will examine whether the articles apply AI to computed tomography (CT) data. Additionally, we aim to determine whether the clinical application of AI in each study focuses on diagnosis, prognosis, treatment, or a combination of these.

In [4]:
data = pd.read_csv("data.csv")
data

,Title,Abstract,Year of publication,DOI,Clinical Application,is_CT
0,(18)F-FDG PET/CT Uptake Classification in Lymp...,Background Fluorine 18 ((18)F)-fluorodeoxygluc...,2020,10.1148/radiol.2019191114,diagnosis,True
1,(18)F-FDG-PET/CT Whole-Body Imaging Lung Tumor...,Under the background of (18)F-FDG-PET/CT multi...,2021,10.1155/2021/8865237,diagnosis,True
2,[(123)I]Metaiodobenzylguanidine (MIBG) Cardiac...,PURPOSE: To provide reliable and reproducible ...,2020,10.1007/s11307-019-01406-6,diagnosis,False
3,[Formula: see text]: deep learning-based radio...,Hand-crafted radiomics has been used for devel...,2020,10.1038/s41598-020-69106-8,prognosis,True
4,3-D Convolutional Neural Networks for Automati...,Deep two-dimensional (2-D) convolutional neura...,2019,10.1109/jbhi.2018.2879449,diagnosis,True
...,...,...,...,...,...,...
1117,Weakly unsupervised conditional generative adv...,Because of the rapid spread and wide range of ...,2021,10.1016/j.media.2021.102159,diagnosis,True
1118,Weakly-supervised lesion analysis with a CNN-b...,Objective.Lesions of COVID-19 can be clearly v...,2021,10.1088/1361-6560/ac4316,diagnosis,True
1119,xViTCOS: Explainable Vision Transformer Based ...,"Objective: Since its outbreak, the rapid sprea...",2022,10.1109/jtehm.2021.3134096,diagnosis,True
1120,Fully automatic segmentation of right and left...,Cardiac magnetic resonance imaging (CMR) is a ...,2020,10.1016/j.compmedimag.2020.101786,diagnosis,False


In [5]:
CLINICAL_APPLICATIONS = set(data["Clinical Application"].to_list())
print(f'Clinical Applications: {CLINICAL_APPLICATIONS}')

Clinical Applications: {'combined', 'prognosis', 'treatment', 'diagnosis'}


## Defining a Custom Reviewer

Now let’s define a custom reviewer class, based on which we will create two reviewers: one junior and one senior. These reviewers will analyze the data to determine the clinical application of each article and whether or not the articles involve the application of AI to CT data. Creating a custom reviewer agent class is straightforward—you just need to design a prompt and build your class by inheriting from the basic reviewer class. By doing so, you gain access to all the baseline features, attributes, and methods of the basic reviewer class. For example, as shown below, when defining a senior reviewer, you can provide additional context to enable the senior reviewer to consider the junior reviewer’s assessment of the article and understand why their assistance was requested.

In [6]:
DEFAULT_MAX_RETRIES = 3

generic_prompt = """

**Review the title and abstract below which discuss an article about AI applied to cardiothoracic imaging and answer the following questions:**

---

**Input item:**
<<${item}$>>

---

**Questions**

1. Does this study apply AI to computed tomography imaging? Answer as True or False
2. What is the main clinical application of AI in this study? Choose one of the following: ${clinical_applications}$
3. How much do you need to seek help from a senior reviewer to double-check your answers about this article? 
   Answer with an integer number between 1 to 5 where 1 is not needed at all to 5 is absolutely needed!
4. If you scored your need for help as 3, 4, or 5, explain why. If you scored 1 or 2, leave the "why_help" field blank. 

"""


class AbstractReviewer(BasicReviewer):
    generic_prompt: Optional[str] = generic_prompt
    response_format: Dict[str, Any] = {"is_ct": bool, "clinical_applications": str, "need_help": int, "why_help": str}
    input_description: str = "article title/abstract"
    clinical_applications: List[str] = list(CLINICAL_APPLICATIONS)
    max_retries: int = DEFAULT_MAX_RETRIES


Now let's define two imaginary junior and senior reviewer agents that both belong to the AbstractReviewer class.

In [7]:
Albert = AbstractReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="Albert",
    max_concurrent_requests=3, 
    backstory="a postdoc researcher with background in AI and medical imaging, with a very skeptical and critical mind",
    model_args={"max_tokens": 200, "temperature": 0.9},
)

Julia = AbstractReviewer(
    provider=LiteLLMProvider(model="gpt-4o"),
    name="Julia",
    max_concurrent_requests=3, 
    backstory="a senior cardiothoracic radiologist with a PhD in computer science and years of research experience in radiology, AI, and data science",
    model_args={"max_tokens": 200, "temperature": 0.1},
    additional_context="""
    A student reviewer has already reviewed this article and needs your help to double-check their answers. 
    Look above to see their answers and the questions they had before providing your own answers. 
    If you are in doubt as well, ask for help and explain why.
    """
)

In [9]:
Albert.formatted_prompt

'**Review the title and abstract below which discuss an article about AI applied to cardiothoracic imaging and answer the following questions:** --- **Input item:** <<${item}$>> --- **Questions** 1. Does this study apply AI to computed tomography imaging? Answer as True or False 2. What is the main clinical application of AI in this study? Choose one of the following: [\'combined\', \'prognosis\', \'treatment\', \'diagnosis\'] 3. How much do you need to seek help from a senior reviewer to double-check your answers about this article? Answer with an integer number between 1 to 5 where 1 is not needed at all to 5 is absolutely needed! 4. If you scored your need for help as 3, 4, or 5, explain why. If you scored 1 or 2, leave the "why_help" field blank.'

## Review

Finally, let's define a review workflow as usual and do our review in two rounds. We will ask the senior reviewer to only look at the articles for which the junior reviewer reported a score of 3 or higher for needing help.

In [8]:
title_abs_review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [Albert],
            "text_inputs": ["Title", "Abstract"]
        },
        {
            "round": 'B',
            "reviewers": [Julia],
            "text_inputs": ["Title", "Abstract", "round-A_Albert_output"],
            "filter": lambda row: int(row["round-A_Albert_need_help"]) > 2
        }
    ]
)

# Reload the data if needed.
data_sample = data.sample(30)
data_sample.reset_index(inplace=True)
updated_data = asyncio.run(title_abs_review(data_sample))

print("\n====== Costs ======\n")
print("Total cost: ", title_abs_review.get_total_cost())
print("Detailed costs: ", title_abs_review.reviewer_costs)

updated_data


====== Starting review round A (1/2) ======

Processing 30 eligible rows


['round: A', 'reviewer_name: Albert'] -                     2025-01-04 21:14:52: 100%|██████████| 30/30 [00:11<00:00,  2.69it/s]


The following columns are present in the dataframe at the end of Albert's reivew in round A: ['index', 'Title', 'Abstract', 'Year of publication', 'DOI', 'Clinical Application', 'is_CT', 'round-A_Albert_output', 'round-A_Albert_is_ct', 'round-A_Albert_clinical_applications', 'round-A_Albert_need_help', 'round-A_Albert_why_help']

====== Starting review round B (2/2) ======

Processing 2 eligible rows


['round: B', 'reviewer_name: Julia'] -                     2025-01-04 21:15:04: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]

The following columns are present in the dataframe at the end of Julia's reivew in round B: ['index', 'Title', 'Abstract', 'Year of publication', 'DOI', 'Clinical Application', 'is_CT', 'round-A_Albert_output', 'round-A_Albert_is_ct', 'round-A_Albert_clinical_applications', 'round-A_Albert_need_help', 'round-A_Albert_why_help', 'round-B_Julia_output', 'round-B_Julia_is_ct', 'round-B_Julia_clinical_applications', 'round-B_Julia_need_help', 'round-B_Julia_why_help']

====== Costs ======

Total cost:  0.00342075
Detailed costs:  {('A', 'Albert'): 0.00015075, ('B', 'Julia'): 0.0032700000000000003}


,index,Title,Abstract,Year of publication,DOI,Clinical Application,is_CT,round-A_Albert_output,round-A_Albert_is_ct,round-A_Albert_clinical_applications,round-A_Albert_need_help,round-A_Albert_why_help,round-B_Julia_output,round-B_Julia_is_ct,round-B_Julia_clinical_applications,round-B_Julia_need_help,round-B_Julia_why_help
0,646,Exploiting Global Structure Information to Imp...,"In this paper, we propose a method to enhance ...",2021,10.3390/s21093249,diagnosis,False,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None
1,1077,Truncated inception net: COVID-19 outbreak scr...,"Since December 2019, the Coronavirus Disease (...",2020,10.1007/s13246-020-00888-x,diagnosis,False,"{'is_ct': False, 'clinical_applications': 'dia...",False,diagnosis,2,,None,None,None,None,None
2,779,Lungs nodule detection framework from computed...,The emergence of cloud infrastructure has the ...,2019,10.1002/jemt.23275,diagnosis,True,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None
3,202,Augmenting existing deterioration indices with...,"IMPORTANCE: When hospitals are at capacity, ac...",2022,10.1371/journal.pone.0263922,diagnosis,False,"{'is_ct': False, 'clinical_applications': 'pro...",False,prognosis,2,,None,None,None,None,None
4,907,Prediction of Pulmonary Fibrosis Based on X-Ra...,"As a fatal lung disease, pulmonary fibrosis ca...",2022,10.1155/2022/3845008,diagnosis,False,"{'is_ct': False, 'clinical_applications': 'dia...",False,diagnosis,2,,None,None,None,None,None
5,216,Automated detection of pulmonary nodules in PE...,PURPOSE: Automated detection of solitary pulmo...,2016,10.1118/1.4948498,diagnosis,True,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None
6,169,Any unique image biomarkers associated with CO...,OBJECTIVE: To define the uniqueness of chest C...,2020,10.1007/s00330-020-06956-w,diagnosis,True,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None
7,647,Exploiting Multiple Optimizers with Transfer L...,Due to the rapid spread of COVID-19 and its in...,2020,10.1155/2020/8889412,diagnosis,False,"{'is_ct': False, 'clinical_applications': 'dia...",False,diagnosis,3,The application of AI techniques in medical im...,"{'is_ct': False, 'clinical_applications': 'dia...",False,diagnosis,3,The study involves complex AI methodologies ap...
8,39,A deep learning algorithm using CT images to s...,OBJECTIVE: The outbreak of Severe Acute Respir...,2021,10.1007/s00330-021-07715-1,diagnosis,True,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None
9,417,COVID-19 lung infection segmentation with a no...,With the global outbreak of COVID-19 in early ...,2021,10.1016/j.media.2021.102205,diagnosis,True,"{'is_ct': True, 'clinical_applications': 'diag...",True,diagnosis,2,,None,None,None,None,None


We can also inspect the indiviual rows of our new dataframe to see the outputs for both reviewers. 

In [11]:
index = 3 # Choose a row of interest in the updated_data dataframe
for key in updated_data.columns:
    if "round" in key:
        print(key, updated_data.iloc[index][key])

round-A_Albert_output {'is_ct': False, 'clinical_applications': 'diagnosis', 'need_help': 3, 'why_help': "I am somewhat confident in my answers, but I'd like to verify the nuances of how AI is integrated with knowledge-driven reasoning in this context, as well as ensure I didn't overlook any significant implications of the methodology."}
round-A_Albert_is_ct False
round-A_Albert_clinical_applications diagnosis
round-A_Albert_need_help 3
round-A_Albert_why_help I am somewhat confident in my answers, but I'd like to verify the nuances of how AI is integrated with knowledge-driven reasoning in this context, as well as ensure I didn't overlook any significant implications of the methodology.
round-B_Julia_output {'is_ct': False, 'clinical_applications': 'diagnosis', 'need_help': 2, 'why_help': ''}
round-B_Julia_is_ct False
round-B_Julia_clinical_applications diagnosis
round-B_Julia_need_help 2
round-B_Julia_why_help 
